In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
gdb_path = '../data/soil/extract/gSSURGO_WA.gdb'

vine_path = '../data/polygons/RegressionRidge.geojson'
vine_poly = gpd.read_file(vine_path)

vine_smol_path = '../data/polygons/RegressionRidge_smol_smol.pkl'
vine_smol_poly = pd.read_pickle(vine_smol_path)

In [ ]:
vine_poly = vine_poly.to_crs('EPSG:5070')
vine_smol_poly = vine_smol_poly.to_crs('EPSG:5070')

bbox = tuple(vine_poly.total_bounds)

soils_subset = gpd.read_file(
    gdb_path,
    layer='MUPOLYGON',
    bbox=bbox
)

soils_clipped = gpd.clip(soils_subset, vine_poly)
# soils_clipped.to_file('soils_vineyard.gpkg', driver='GPKG')

In [ ]:
soil_exploded = soils_clipped.explode(index_parts=False)

In [ ]:
soil_clipped_to_vine = gpd.clip(soil_exploded, vine_poly)

In [ ]:
soil_clipped_to_plot = gpd.clip(soil_exploded, vine_smol_poly)

In [ ]:
soil_final = soils_clipped.dissolve(by='MUKEY', as_index=False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
target_crs = 'EPSG:32610'

vine_smol_poly = vine_smol_poly.to_crs(target_crs)
soil_proj = soil_clipped_to_plot.to_crs(target_crs)


In [ ]:

fig, ax = plt.subplots(figsize=(10, 6))

# Plot soil polygons colored by MUKEY
soil_proj.plot(
    ax=ax, 
    column='MUKEY',
    cmap='Set3',
    edgecolor='black',
    alpha=0.7,
    linewidth=0,
    legend=True,
    legend_kwds={'title': 'MUKEY'}
)

# Plot vineyard outline on top
vine_smol_poly.plot(
    ax=ax,
    color='none',
    edgecolor='red',
    linewidth=0
)
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.xticks([])
plt.yticks([])


plt.title('Regression Ridge\nUSGS Soil map')
# Grab the legend object and set its location
leg = ax.get_legend()
if leg:
    leg.set_bbox_to_anchor((0.17, 1))  # move it outside top-right corner
    # or use standard locations like 'upper left', 'lower right', etc.
    # leg.set_loc('upper left')  # only works in newer Matplotlib

    
plt.savefig('../img/soil.png')
plt.show()


In [ ]:
import fiona
import pandas as pd
pd.set_option('display.max_columns', None)
def read_table_subset(gdb_path, layer_name, filter_keys = None, key_column = None):
    '''
    Read a non-spatial layer from a .gdb into a Pandas DataFrame,
    optionally keeping only rows where key_column is in filter_keys.
    '''
    with fiona.open(gdb_path, layer = layer_name) as src:
        if filter_keys is None:
            df = pd.DataFrame([f['properties'] for f in src])
        else:
            # Only include rows where key_column is in filter_keys
            records = [f['properties'] for f in src if f['properties'][key_column] in filter_keys]
            df = pd.DataFrame(records)
    return df

In [ ]:
vineyard_mukeys = soils_clipped['MUKEY'].unique()

components_vineyard = read_table_subset(
    gdb_path,
    'component',
    filter_keys = vineyard_mukeys,
    key_column = 'mukey'
)

In [ ]:
cokeys = components_vineyard['cokey'].unique()

horizons_vineyard = read_table_subset(
    gdb_path,
    'chorizon',
    filter_keys = cokeys,
    key_column = 'cokey'
)

In [ ]:
chkeys = horizons_vineyard['chkey'].unique()

textures_vineyard = read_table_subset(
    gdb_path,
    'chtexturegrp',
    filter_keys = chkeys,
    key_column = 'chkey'
)

In [ ]:
# Merge horizons → textures
horizons_vineyard = horizons_vineyard.merge(
    textures_vineyard,
    on="chkey",
    how="left",
    suffixes=("", "_tex")
)

# Merge horizons → component
horizons_vineyard = horizons_vineyard.merge(
    components_vineyard,
    on="cokey",
    how="left",
    suffixes=("", "_comp")
)

In [ ]:
non_cols = [col for col in horizons_vineyard.columns if horizons_vineyard[col].unique().any() == None]

horizons_vineyard = horizons_vineyard.drop(columns = non_cols)

# drop the non-dominant soil layer
horizons_vineyard = horizons_vineyard.drop(horizons_vineyard[horizons_vineyard['mukey'] == '69508'].index)

# drop the unweathered bedrock outcrop
horizons_vineyard = horizons_vineyard.drop(horizons_vineyard[horizons_vineyard['cokey'] == '25272336'].index)
horizons_vineyard = horizons_vineyard.drop(horizons_vineyard[
    horizons_vineyard['texdesc'].str.contains('bedrock', case=False, na=False)
].index)



In [ ]:
single_value_cols = horizons_vineyard.columns[horizons_vineyard.nunique() == 1].tolist()

horizons_vineyard = horizons_vineyard.drop(columns = single_value_cols)

# Drop columns not needed for numeric analysis
cols_to_drop = ['chtgkey', 'rvindicator', 'texdesc', 'texture']
horizons_vineyard = horizons_vineyard.drop(columns=cols_to_drop)

# Drop duplicate horizons (same component and depth)
# horizons_numeric = horizons_numeric.drop_duplicates(subset=['cokey', 'hzdept_h', 'hzdepb_h'])


In [ ]:
horizons_vineyard = horizons_vineyard.drop_duplicates()

In [ ]:
horizons_vineyard = horizons_vineyard.drop(horizons_vineyard[horizons_vineyard['hzname'] == 'H3'].index)

In [ ]:
horizons_vineyard = horizons_vineyard.reset_index(drop = True)

In [ ]:
horizons_vineyard

In [ ]:
import numpy as np
def summarize_horizons(df):
    df = df.copy()

    # --- Compute horizon thickness ---
    df['thickness'] = df['hzdepb_r'] - df['hzdept_r']

    # --- Force numeric columns (convert bad strings to NaN) ---
    numeric_cols = [
        'sandtotal_r', 'silttotal_r', 'claytotal_r',
        'ph1to1h2o_r', 'cec7_r', 'om_r',
        'caco3_r', 'dbovendry_r', 'kffact', 'kwfact',
        'frag3to10_r', 'fraggt10_r',
        'awc_r', 'ec_r',
        'wthirdbar_r', 'wtenthbar_r', 'wfifteenbar_r',
        'wsatiated_r'
    ]
    for col in numeric_cols + ['hzdept_r', 'hzdepb_r', 'thickness']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- Define weighted mean function (safe) ---
    def weighted_mean(x):
        w = df.loc[x.index, 'thickness']
        valid = x.notna() & w.notna() & (w > 0)
        if valid.sum() == 0:
            return np.nan
        return np.average(x[valid], weights=w[valid])

    # --- Summarize by map unit ---
    summary = df.groupby('mukey').agg({
        # --- Texture & composition ---
        'sandtotal_r': weighted_mean,
        'silttotal_r': weighted_mean,
        'claytotal_r': weighted_mean,
        'frag3to10_r': weighted_mean,
        'fraggt10_r': weighted_mean,
        'awc_r': weighted_mean,
        'cec7_r': weighted_mean,
        'om_r': weighted_mean,
        'caco3_r': weighted_mean,

        # --- Soil physics ---
        'dbovendry_r': weighted_mean,
        'wthirdbar_r': weighted_mean,
        'wtenthbar_r': weighted_mean,
        'wfifteenbar_r': weighted_mean,
        'wsatiated_r': weighted_mean,

        # --- Chemistry ---
        'ph1to1h2o_r': weighted_mean,
        'ec_r': weighted_mean,

        # --- Erosion & permeability factors ---
        'kffact': 'mean',
        'kwfact': 'mean',

        # --- Depth ---
        'thickness': 'sum',    # total profile depth
        'hzdepb_r': 'max'      # deepest horizon boundary
    }).reset_index()

    # --- Rename ---
    summary.rename(columns={
        'thickness': 'profile_depth',
        'hzdepb_r': 'max_depth'
    }, inplace=True)

    return summary


In [ ]:
hv = summarize_horizons(horizons_vineyard)

In [ ]:
hv

In [ ]:
target_crs = 'EPSG:5070'

vine_smol_poly = vine_smol_poly.to_crs(target_crs)
soil_proj = soil_clipped_to_plot.to_crs(target_crs)


In [ ]:
df = pd.merge(hv, soil_proj, how = 'inner', left_on = 'mukey', right_on = 'MUKEY')

In [ ]:
vine_smol_poly['plot_id'] = vine_smol_poly.index

In [ ]:
# Intersect polygons — this can be slow for large datasets, but precise
overlaps = gpd.overlay(vine_smol_poly, soil_proj, how='intersection')

# Compute intersection area in consistent units (e.g., m²)
overlaps['area_overlap'] = overlaps.geometry.area

# Compute area fractions relative to each plot
plot_areas = vine_smol_poly[['plot_id', 'geometry']].copy()
plot_areas['plot_area'] = plot_areas.geometry.area

# Merge areas back to overlaps
overlaps = overlaps.merge(plot_areas[['plot_id', 'plot_area']], on='plot_id')
overlaps['area_fraction'] = overlaps['area_overlap'] / overlaps['plot_area']


In [ ]:
overlaps = overlaps.merge(
    hv[[
        'sandtotal_r', 'silttotal_r', 'claytotal_r', 
        'awc_r', 'cec7_r', 'om_r', 
        'ph1to1h2o_r', 'ec_r',
        'profile_depth', 'max_depth','mukey',
        'frag3to10_r', 'fraggt10_r', 'dbovendry_r', 'caco3_r'
    ]],
    right_on='mukey',
    left_on = 'MUKEY',
    how='left'
)


In [ ]:
def area_weighted_mean(x, weights):
    valid = x.notna() & weights.notna()
    if valid.sum() == 0:
        return np.nan
    return np.average(x[valid], weights=weights[valid])

weighted_cols = ['sandtotal_r', 'silttotal_r', 'claytotal_r', 
        'awc_r', 'cec7_r', 'om_r', 
        'ph1to1h2o_r', 'ec_r',
        'profile_depth', 'max_depth',
        'frag3to10_r', 'fraggt10_r', 'dbovendry_r', 'caco3_r']

plot_soil_summary = (
    overlaps.groupby('plot_id')
    .apply(lambda g: pd.Series({
        col: area_weighted_mean(g[col], g['area_fraction'])
        for col in weighted_cols
    }))
    .reset_index()
)


In [ ]:
plot_soil_summary = pd.DataFrame(plot_soil_summary)

In [ ]:
plot_soil_summary

In [ ]:
plot_soil_summary.to_pickle('../data/soil/plot_summary.pkl')